In [1]:
import numpy as np

import sys
sys.path.append('/Users/adityasyam/grg_pheno_sim') 

from grg_pheno_sim.multi_grg_phenotype import sim_phenotypes_multi_grg
from grg_pheno_sim.model import grg_causal_mutation_model


The following commands only serve the purpose of converting the VCF zip file into GRGs that will be used for phenotype simulation.

In [2]:
%%script bash --out /dev/null
if [ ! -f test-200-samples.grg ]; then
  grg construct --no-maf-flip -p 10 -t 2 ../data/test-200-samples.vcf.gz --out-file test-200-samples.grg
fi

In [3]:
%%script bash --out /dev/null
if [ ! -f test-200-samples_copy.grg ]; then
  grg construct --no-maf-flip -p 10 -t 2 ../data/test-200-samples.vcf.gz --out-file test-200-samples_copy.grg
fi

In [4]:
grg_list = ["test-200-samples.grg", "test-200-samples_copy.grg"]
#this is the list of GRG files to be loaded in 

We will first demonstrate loading all GRG files into RAM and simulating phenotypes. Causal mutations are sampled from each GRG, and the genetic values are obtained for the samples. The combined genetic dataframe is the addition of each GRG's genetic values (for each causal mutation ID). Noise is sampled at the end and added to obtain the phenotypes.

NOTE: It is necessary for each GRG to have the same number of samples.

In [5]:
model_type = "multivariate normal"
means = np.zeros(2)
cov = np.array([[1, 0.25], [0.25, 1]])

model = grg_causal_mutation_model(model_type, mean=means, cov=cov)

num_causal_per_file = 1000

random_seed = 1

normalize_genetic_values_before_noise = False

noise_heritability = [0.33, 0.25]

load_all_into_RAM = True #this parameter decides whether to load all GRGs into RAM together

multi_grg_multi_phenotypes = sim_phenotypes_multi_grg(grg_list, model, num_causal_per_file, random_seed, load_all_ram=load_all_into_RAM, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, noise_heritability=noise_heritability)


Loaded test-200-samples.grg into RAM
Loaded test-200-samples_copy.grg into RAM
Genetic values for test-200-samples.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0     -29.245763                   0
1                0     -17.120606                   1
2                1     -48.676841                   0
3                1      -8.338623                   1
4                2     -27.383382                   0
..             ...            ...                 ...
395            197      -2.096535                   1
396            198     -40.165943                   0
397            198      -0.515813                   1
398            199     -34.147763                   0
399            199       1.103035                   1

[400 rows x 3 columns]
Genetic values for test-200-samples_copy.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0       4.602603                   0
1                0      

In [6]:
multi_grg_multi_phenotypes

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,-0.146220,-0.882502,-1.028722
1,1,0,-0.337850,0.397966,0.060116
2,0,1,-0.036993,1.078081,1.041088
3,1,1,0.507905,-0.149452,0.358453
4,0,2,0.025584,2.128519,2.154103
...,...,...,...,...,...
395,1,197,-0.555971,-0.524712,-1.080683
396,0,198,0.073651,1.275030,1.348681
397,1,198,1.138926,1.446696,2.585622
398,0,199,-0.517340,-1.148811,-1.666151


We now perform similar simulations, but by loading the GRGs into RAM sequentially (instead of all together).

In [7]:
model_type = "multivariate normal"
means = np.zeros(2)
cov = np.array([[1, 0.25], [0.25, 1]])

model = grg_causal_mutation_model(model_type, mean=means, cov=cov)

num_causal_per_file = 1000

random_seed = 1

normalize_genetic_values_before_noise = False

noise_heritability = [0.33, 0.25]

load_all_into_RAM = False #this parameter decides whether to load all GRGs into RAM together

multi_grg_multi_seq_phenotypes = sim_phenotypes_multi_grg(grg_list, model, num_causal_per_file, random_seed, load_all_ram=load_all_into_RAM, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, noise_heritability=noise_heritability)


Loaded test-200-samples.grg into RAM
Genetic values for test-200-samples.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      -2.136211                   0
1                0      25.389768                   1
2                1       3.481988                   0
3                1      20.545445                   1
4                2      -9.922663                   0
..             ...            ...                 ...
395            197      17.404438                   1
396            198     -13.714414                   0
397            198      10.695622                   1
398            199       3.172003                   0
399            199      17.567172                   1

[400 rows x 3 columns]
Loaded test-200-samples_copy.grg into RAM
Genetic values for test-200-samples_copy.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0     -13.036428                   0
1                0     -

In [8]:
multi_grg_multi_seq_phenotypes

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,-0.334113,0.990788,0.656675
1,1,0,0.255776,-0.122345,0.133431
2,0,1,-0.207421,-0.975096,-1.182517
3,1,1,0.065537,-1.035354,-0.969817
4,0,2,0.112398,0.751293,0.863691
...,...,...,...,...,...
395,1,197,0.152377,0.535378,0.687756
396,0,198,-0.388314,0.750428,0.362114
397,1,198,0.099092,-1.515798,-1.416706
398,0,199,0.665796,0.947686,1.613482
